# AutoEncoder

## I. NN中的layer的功能：以分类任务为例

### I.1 NN基础功能模块的工作机制
1. <font color=blue>**activation function做为threshold gate控制信息从浅层向深层传递。**</font>
   - 比如：典型的ReLU设置的threshold为0，只有$X@W>0$的时候，W所代表的feature才被activated。
   - 当matrix上的elements value超过这个threshold的时候，它的值才会向更深层的网络传递。
2. <font color=blue>**inner product做feature match，只有与layer中的weights match度高的信息才可能通过activation function设置的threshold。**</font>
   - inner product的计算特征：如果向量x与向量w的长度固定，那么他们之间的夹角越小时，x@w的值越大
     - 根据<font color=norange>**the curse of dimensionality**</font>,当向量的维度较大时，randomly chosen vectors的长度会集中在其均值的附近。<font color=brown>[详见curse of dimensionality的笔记]</font>因此，在高维情景下，x与w的长度也都可以视为在一定范围内固定。
     - 所以，x@w的值由他们的夹角$\theta$对应的$cos\theta$决定。$\theta$越小，此时x与w越接近，x@w的值越大。
   - 因此，affine layer的output体现的就是x与w的match程度。<font color=green>可以将weights视为feature template，一旦x向量中的信息与该feature接近，那么对应的output值就大。只要超过了activation function设置的threshold，这个信息就会向更深层的网络传递。</font>
   - <font color=brown>注意：weights代表feature templates是一种抽象的理解方式。feature具体是什么很难直观理解。在CNN中，第一层layer得到的feature可以画图来看。实际得到的图像结果也印证了浅层feature是简单的基础视觉元素的观点。</font>

3. <font color=blue>**loss function既引导weights迭代，又通过loss大小衡量得到的weights(feature)的质量。**</font>
   - 在C分类问题中，last affine layer是linear classifier。它有C个neuron，每个对应一个类型。实现min loss等价于实现让每个样本在其类型对应的neuron上有更高的score，而在其他位置处有更低的score。
   - linear classifier的input是倒数第二个affine layer的feature match结果经过activation function的gate之后的output。
   - 如果linear classifier得到的input**性质很好**，也就是它得到了能差异化各个类型的features，那么linear classifier就更容易完成分类任务。所以feature提取的结果会决定目标函数最终的优化结果。
   - feature就是网络中的weights。目标函数既引导着weights的迭代，又通过loss大小衡量weights(feature)的质量。唯一作为外部因素影响weights质量的是网络结构。
   - <font color=red>注意，如果linear classifier得到的input是线性可分，linear classifier+softmax不能保证100%正确分类，这点和perceptron不同。</font>
4. <font color=blue>**back propagation + gradient Descent作为最优化的基本方法，帮助网络找到更好的weights来实现min loss的目标。**</font>
   - 提供了weight的迭代优化路径，让weight的迭代更新朝着使loss越来越小的方向发展。
   - 最后会收敛到局部最优解。
5. <font color=blue>**网络结构决定了weights在哪里搭templates，templates与templates之间如何合作来搭建复杂的feature templates。**</font>
   - 在整个网络迭代过程中，一旦网络结构确定下来，那么weights是在loss function的引导下被动变化的。所以，在优化方法对结果影响不大的条件下，找到好的weights(feature template)靠的是loss function和网络结构。

### I.2 NN各层在做什么
1. **最后一层affine layer + softmax是linear classifier。**但并不是一定要用affine+softmax，用affine layer + svm，甚至不用affine，直接用global average pool等在feature的基础上做classifier
2. **从第一层到倒数第二层的layer都在做"feature extraction"。**目的是将data从原始数据转变成linearly seperable features。
   - 以数字识别为例
     - 最浅层的layer学到了digits的图片中的significant feature，还有训练集中非digits的那些图片的significant feature，后者用来判断非digits时会用到。
     - 在更深的layer中，会将浅层layer的feature做construct，最后在倒数第2层上获得能直接判断是digits或者不是digits的feature。
<img src="../pics/feature_extraction.png" width="300">

- <font color=deeppink>**而loss function和网络结构的设计目标都是**：让网络通过GD迭代得到的各层features汇集到倒数第二层上时得到的feature是尽可能linearly separable的。</font>

### I.3 feature extraction能力的延伸
1. 如果lower layer中提取到的feature足够表达inputs的特征。比如上面以digits信号作为输入，提取到了下图中的各个local feature。那么理论上，更深层的网络应该可以用这些feature重构原图。
<img src="../pics/local_feature.png" width="360" height="300">
2. 但是上面分类任务训练的网络做不到，因为它的目标函数设定是让网络识别input是不是digits，所以loss function引导的优化任务会让网络只保留那些能Discriminate digits和non-digit的feature，其他信息网络上不会保留。
   - 比如，如果大多数non-digit inputs中都有 <img src="../pics/non_digit.png" style="display:inline;height:20px;"> feature，而在digits中这个feature也基本上都存在，那这个feature对于分类任务来说就是non-discriminative的。网络很可能不会保留这个feature。但对于reconstruct digit来说，这个feature是很重要的。
3. 重新设计学习任务，让网络learn feature然后重构自己，这种网络称为autoencoder(AE).

## II. AutoEncoder

### II.1 模型的思路和工作方式
- **思路**：
  - 用encoder-decoder结构来让output复现input。如果利用学到的feature能较好地重建input，说明feature很好地表达了input中的关键信息。

- **类型**：
  - 一种从raw data中learn feature的无监督学习方法。
  - AE不涉及概率，所以不是概率model

- **工作方式**：
  - 在training阶段通过$min \begin{Vmatrix}\hat x - x\end{Vmatrix}^2_2$得到encoder和decoder中的weights
    <img src="../pics/ae.png" width="360" height="300">
  - 完成training后扔掉decoder，保留encoder作为特征提取器在下游任务中使用
     - 在分类任务中，把encoder接上classifier，再将encoder和classifier一起finetune
     - 在image caption中，接上生成文字的decoder后再一起finetune。


- **要点**：feature z的维度要比input的维度低。
  - 如果是做compression，feature z的维度自然需要比x低。
  - 理论上如果是做feature extraction，特征本身的数量很可能超过单个sample的dimension。假如input的sample量很大，那么也可以控制overfit。但实际上在AE训练中，feature z的dim设置还是要小于input dim。因为放开z的维度控制的话，很可能训练中拿到的encoder和decoder都只是identity function，网络本身没有真正的feature extract能力。

### II.2 AE的能力和局限性
#### II.2.1 从PCA的角度理解AE的compression能力
1. **与PCA等价的AE**
   - 设计：如果AE的encoder和decoder都只有一层，且encoder上那一层affine layer的activation function是identity function，那么该AE就是一个PCA
     - encoder：$z=f(w_{en}x)=w_{en}x$
     - decoder：$\hat x = w_{de}z = w_{de}w_{en}x$
     - loss func: $min \begin{Vmatrix}\hat x - x\end{Vmatrix}^2_2$和PCA中的优化目标一样。
   - 说明：
     - identity function是linear function，且encoder的output维度比input维度低，所以encoder是把input投影到了超平面上。
   - 示例：
     1. dim(z)=1
     <img src="../pics/pca_1D.png" width="360" height="300">
     2. dim(z)=2
     <img src="../pics/pca_2D.png" width="160">

2. **与non-linear PCA等价的AE**
   - 设计：和linear中一样，只是把activation function换成non-linear的
   - 示例：dim(z)=1
     <img src="../pics/pca_non_linear.png" width="160">

3. **从浅层网络扩展到深度网络**
   - 设计：在前两中等价于PCA的网络中，encoder和decoder的网络都只有一层。如果把网络加深，那么它们就能表达更复杂的manifolds
   - 示例：dim(z)=1,layer两层，每层100个neuron。
   <img src="../pics/ae_2layer.png" width="400">
   

### II.3 扩展
#### II.3.1 思路：用AE的decoder做generator
- AE中encoder做feature extraction，decoder做input reconstruction。既然encoder可以单独当做特征提取器使用，那么decoder应该能做generator。
#### II.3.2 用AE的decoder做generator的局限性
1. <font color=brown>[经验结论]</font>AE得到的decoder做为generator的泛化能力有限。
   - 在上面dim(z)=1的例子中，很容易给decoder一个初始值。但一旦这个初始值超出模型在training set中见过的范围的值，得到的形态不能体现出原图中的螺旋状特征。
   - 所以如果要用decoder部分做generator只能给z取值在training set中出现过的z的范围。
2. 而在复杂的生成任务中，比如人脸生成任务，dim(z)是高维的，这时候如何给出z的合理初始值本身也成了问题。
   - 因为在AE的训练中，并没有给出z的分布，也就是说无法通过sample方式给decoder的input给出合适的初始值来完成生成任务。
   - 而如果初始化的值不合理，又会因为decoder泛化能力的约束导致生成的内容与training set中的样本特征差异较大。
   - <font color=orange>后来VAE模型解决了这个问题。</font>